In [1]:
import pandas as pd
from pycaret.classification import *
import os

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
screenshots = pd.read_csv("images/screenshots/screenshots.csv")

In [4]:
regulars = pd.read_csv("images/regular/regular.csv")

In [5]:
not_good = pd.read_csv("images/not_good/not_good.csv")

In [6]:
superb = pd.read_csv("images/superb/superb.csv")

In [7]:
docs = pd.read_csv("images/docs/docs.csv")

In [8]:
test = pd.read_csv("images/test/test.csv")

In [9]:
regulars["screenshots"] = 0
not_good["screenshots"] = 0
superb['screenshots'] = 0
screenshots["screenshots"] = 1
docs["screenshots"] = 0

In [10]:
whole_docs = pd.concat([regulars, not_good, superb, screenshots, docs], ignore_index=True)

In [11]:
whole_docs = whole_docs.drop(columns=["filename", "depth"])
whole_docs.sample()

,size_w,size_h,ch,aspect_ratio,mean_value_b,mean_value_g,mean_value_r,median_value_b,median_value_g,median_value_r,std_deviation_b,std_deviation_g,std_deviation_r,contrast,brightness,hog_features_mean,hog_features_median,hog_features_std,hog_features_len,h_mean,h_median,h_std,s_mean,s_median,s_std,v_mean,v_median,v_std,n_objects,area,area_mean,area_std,vert,vert_mean,vert_std,color_mean,color_std,corners,lines,faces,has_text,filesize,sharpness,gradient,texture_energy,texture_contrast,color_balance,focus_score,screenshots
685,4032,3024,3,0.75,181.086469,185.598769,199.403174,185.0,190.0,204.0,24.753855,26.67149,26.919523,26.305908,189.247468,0.033812,0.037106,0.010331,1524096,27.026543,10.0,50.281775,27.872467,24.0,17.37754,200.442431,204.0,22.773244,245,12141094.5,49555.487755,773317.685457,539,2.2,1.808371,129.876695,6.111128,100,8426,0,1,3801179,93.44495,1557.304537,0.001938,0.0,26.114956,1557.304537,0


In [12]:
corr = whole_docs.corr()


In [13]:
corr.screenshots


size_w                -0.445845
size_h                -0.438040
ch                          NaN
aspect_ratio           0.135260
mean_value_b           0.415279
mean_value_g           0.415632
mean_value_r           0.341514
median_value_b         0.432510
median_value_g         0.441529
median_value_r         0.355364
std_deviation_b        0.143720
std_deviation_g        0.121104
std_deviation_r        0.168979
contrast               0.136617
brightness             0.404970
hog_features_mean     -0.604491
hog_features_median   -0.554497
hog_features_std       0.282990
hog_features_len      -0.433820
h_mean                -0.038041
h_median              -0.018578
h_std                  0.025440
s_mean                -0.276420
s_median              -0.293105
s_std                  0.090138
v_mean                 0.371063
v_median               0.385070
v_std                 -0.048055
n_objects             -0.180575
area                  -0.326676
area_mean              0.097896
area_std

In [14]:
s = setup(whole_docs, target = 'screenshots', session_id = 123)

,Description,Value
0,Session id,123
1,Target,screenshots
2,Target type,Binary
3,Original data shape,"(731, 49)"
4,Transformed data shape,"(731, 49)"
5,Transformed train set shape,"(511, 49)"
6,Transformed test set shape,"(220, 49)"
7,Numeric features,48
8,Rows with missing values,1.5%
9,Preprocess,True


In [15]:
best_not_good = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9549,0.9873,0.7864,0.9889,0.8668,0.8412,0.8547,0.0240
lightgbm,Light Gradient Boosting Machine,0.9529,0.9834,0.8173,0.9476,0.8681,0.8407,0.8499,0.2210
catboost,CatBoost Classifier,0.9491,0.9812,0.7882,0.9609,0.8555,0.8260,0.8384,1.4780
rf,Random Forest Classifier,0.9393,0.9780,0.7291,0.9675,0.8205,0.7862,0.8040,0.0400
ada,Ada Boost Classifier,0.9374,0.9762,0.8564,0.8505,0.8458,0.8069,0.8127,0.0270
gbc,Gradient Boosting Classifier,0.9373,0.9777,0.7945,0.8869,0.8326,0.7944,0.8002,0.0850
xgboost,Extreme Gradient Boosting,0.9353,0.9761,0.7864,0.8902,0.8234,0.7851,0.7945,0.0240
qda,Quadratic Discriminant Analysis,0.9197,0.9424,0.7000,0.9050,0.7672,0.7220,0.7440,0.0060
ridge,Ridge Classifier,0.9178,0.0000,0.6509,0.9214,0.7504,0.7046,0.7271,0.0060
lda,Linear Discriminant Analysis,0.9139,0.9445,0.6718,0.8854,0.7485,0.6995,0.7187,0.0070


In [16]:
tune_model(best_not_good)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9412,0.9707,0.8000,0.8889,0.8421,0.8061,0.8078
2,0.9608,0.9659,0.8000,1.0000,0.8889,0.8654,0.8734
3,0.9608,0.9927,0.9000,0.9000,0.9000,0.8756,0.8756
4,0.8627,0.8707,0.7000,0.6364,0.6667,0.5805,0.5815
5,0.8824,0.9659,0.8000,0.6667,0.7273,0.6531,0.6575
6,0.9608,0.9854,0.9000,0.9000,0.9000,0.8756,0.8756
7,0.9412,1.0000,1.0000,0.7692,0.8696,0.8324,0.8444
8,0.9804,0.9864,0.9091,1.0000,0.9524,0.9401,0.9418


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=100, n_jobs=-1, oob_score=False,
                     random_state=123, verbose=0, warm_start=False)

In [17]:
ids_test = test.pop("filename")

In [18]:
not_good_pred = predict_model(best_not_good, data=test)

In [19]:
not_good_pred["name"] = ids_test

In [20]:
not_good_photos = not_good_pred.loc[not_good_pred.prediction_label == 1 ,["name", "prediction_label", "prediction_score"]]

In [21]:
if not os.path.isdir("./images/test/1/"):
    os.mkdir("./images/test/1/")
if not os.path.isdir("./images/test/0/"):
    os.mkdir("./images/test/0/") 


In [22]:
def movingImage(filename, location, destination):
    location = location + filename
    destination = destination + filename
    os.rename(location, destination)

In [23]:
for step in not_good_photos["name"]:
    movingImage(step, "./images/test/", "./images/test/1/")

In [24]:
not_bad_photos = not_good_pred.loc[not_good_pred.prediction_label == 0 ,["name", "prediction_label", "prediction_score"]]

In [25]:
for step in not_bad_photos["name"]:
    movingImage(step, "./images/test/", "./images/test/0/")

In [26]:
not_good_pred


,size_w,size_h,ch,aspect_ratio,mean_value_b,mean_value_g,mean_value_r,median_value_b,median_value_g,median_value_r,...,depth,sharpness,gradient,texture_energy,texture_contrast,color_balance,focus_score,prediction_label,prediction_score,name
0,2016,4032,3,2.000000,124.210114,140.406235,155.223267,121.0,137.0,155.0,...,uint8,58.429031,4398.277344,0.002092,0.0,49.799664,4398.277344,0,0.93,2019-06-14 22-03-09.JPG
1,3060,4080,3,1.333333,88.949585,107.602760,115.356812,56.0,98.0,120.0,...,uint8,145.967468,3287.873779,0.001295,0.0,77.030762,3287.873779,0,0.96,2024-01-18 14-43-46.JPG
2,3060,4080,3,1.333333,108.731117,126.959404,109.161751,109.0,130.0,121.0,...,uint8,149.195480,4540.938965,0.000587,0.0,69.229446,4540.938965,0,0.95,2022-10-02 14-52-21.JPG
3,3024,4032,3,1.333333,151.404526,160.070007,163.675659,153.0,161.0,165.0,...,uint8,110.315964,1873.225342,0.001089,0.0,15.650246,1873.225342,0,0.99,2023-12-18 11.43.21.JPG
4,4032,3024,3,0.750000,125.188385,125.890472,115.325684,119.0,124.0,112.0,...,uint8,64.723251,1838.991577,0.000523,0.0,70.619270,1838.991577,0,0.95,2023-12-29 14.35.30.JPG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,710,1762,3,2.481690,246.849716,246.861755,246.856049,255.0,255.0,255.0,...,uint8,2104.567871,35219.539062,0.581789,0.0,38.565742,35219.539062,1,0.96,2023-11-10_12-30-59.png
80,1128,2010,3,1.781915,249.278397,247.807861,249.600006,255.0,255.0,255.0,...,uint8,200.866531,11279.880859,0.511364,0.0,29.465454,11279.880859,1,0.70,2023-11-14_17-52-22.png
81,1280,720,3,0.562500,55.307808,101.521416,56.142220,19.0,113.0,15.0,...,uint8,31.970362,1544.194214,0.001289,0.0,59.800686,1544.194214,1,0.66,2017-07-05 18-30-23.JPG
82,889,2483,3,2.793026,201.504608,199.670593,217.101379,235.0,236.0,236.0,...,uint8,6408.433594,77612.460938,0.020001,0.0,65.068916,77612.460938,1,0.68,низ_коралл.jpg


In [27]:
evaluate_model(best_not_good)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…